# Recurrent Neural Network (RNN) untuk data sekuensial
### Modifikasi dengan Optimizer **ADAM**
Benediktus Sashenka
10117080
___

In [1]:
from numpy import *   
import numpy as np
from numpy.random import *
import pandas as pd
import pickle
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sb
#sb.set_theme()
sb.set_style("whitegrid")
#sb.set_style("darkgrid")

%matplotlib inline

np.set_printoptions(precision = 3, suppress = True, formatter = {'float':'{:6.5f}'.format})

#Sigmoid function & its derivative
sigmoid = lambda Z: 1/(1+exp(-Z))
dsigmoid = lambda A: A*(1-A)

#ReLU function & its derivative
ReLU  = lambda Z: Z.clip(0)
#Derivative of ReLU function
dReLU = lambda A: (A > 0)*1

#Derivativer of tanh()
dtanh = lambda A: 1-A**2

#Derivative oh arctanh
darctanh = lambda A: 1/(A**2+1)

#Softplus function & its derivative
splus = lambda Z: log(1+exp(Z))
dsplus = lambda A: 1/(1+exp(-A))

linear = lambda X,w,b: X@w+b

"Time step (ts)"
def steps(x, step):   
    obs  = len(x)-step
    xt   = x[:obs,:]
    for i in arange(1,step+1):
        xt = hstack((xt, x[i:obs+i,:]))   
    return xt

## Modifikasi pada kelas RNN

Sama seperti algoritma pada **SGD Dengan Momentum Nesterov (modifikasi dengan optimizer ADAM)**, modifikasi berada pada bagian updating parameter. Parameter yang ada pada algoritma RNN ini ada $W_0$, $b_0$, $W_1$, $b_1$, dan $W_s$ yang akan dibuat momen untuk masing-masing parameter tersebut untuk update parameter.

Lalu karena sudah memakai momentum optimizer ADAM, learning rate yang digunakan tidak terlalu besar yaitu 0.001.

In [2]:
class RNN:
    def __init__(self,x,nh,alpha,epochs): # ada h nodes di dalam hidden layer  hlayers = [7, 3, 34, 89]
        self.Xtrain  = x #input
        self.ytrain  = x[:,-1:] #output     
        self.nh = nh   #number of neurons in hidden layer
        self.α  = alpha
        self.epochs = epochs
        self.Ts = shape(self.Xtrain)[1]-1   # Time-steps
        self.N, no = shape(self.ytrain)     #jumlah observasi (self.N) dan jumlah output
        self.ni    = 1  #jumlah input
        
        "Initial values untuk parameter w and b"
        seed(20201212)
        self.w0 = randn(self.ni,self.nh)
        self.b0 = randn(1,self.nh)
        self.w1 = randn(self.nh,no)
        self.b1 = randn(1,no)
        self.ws = randn(self.nh,self.nh)
        
        "Initial values untuk momen dengan random, momen v harus positif"
        self.mw0 = randn(self.ni,self.nh)
        self.mb0 = randn(1,self.nh)
        self.mw1 = randn(self.nh,no)
        self.mb1 = randn(1,no)
        self.mws = randn(self.nh,self.nh)
        
        self.vw0 = randn(self.ni,self.nh)**2
        self.vb0 = randn(1,self.nh)**2
        self.vw1 = randn(self.nh,no)**2
        self.vb1 = randn(1,no)**2
        self.vws = randn(self.nh,self.nh)**2
        
    def learning(self):
        self.ycap = []
        for i in range(len(self.Xtrain)):
            self.S1 = [zeros((self.ni, self.nh))]  
            self.S2 = []              
            dCdw0 = zeros_like(self.w0)
            dCdb0 = zeros_like(self.b0)
            dCdw1 = zeros_like(self.w1)
            dCdb1 = zeros_like(self.b1)
            dCdws = zeros_like(self.ws)
            
            "Forward propagation in time step"
            for k in range(self.Ts):       
                A0 = self.Xtrain[i,k]
                yk = self.Xtrain[i,k + 1]
            
                # Forward pass, 1st layer
                Z1 = dot(A0,self.w0) + self.b0 + (self.S1[-1]@self.ws) 
                A1 = tanh(Z1)
                self.S1.append(copy(A1))   
            
                # Forward pass, 2nd layer
                Z2 = A1@self.w1 + self.b1
                A2 = sigmoid(Z2)
                self.S2.append(copy(A2))
                
            self.ycap.append(A2[0])
            
            "Backward propagation in time step"
            for k in arange(self.Ts)[::-1]:     
                A0 = self.Xtrain[i,k]
                yk = self.Xtrain[i,k + 1]
            
                # Backprop, 2nd layer
                dCdZ2 = -(yk - self.S2[k]) * dsigmoid(self.S2[k])
                dCdw1 += dot(self.S1[k+1].T,dCdZ2)
                dCdb1 += sum(dCdZ2)
            
                # Backprop, 1st layer
                dCdZ1 = dCdZ2@self.w1.T * dtanh(self.S1[k+1])
                dCdw0 += dot(A0, dCdZ1)
                dCdb0 += sum(dCdZ1)
            
                # Backprob, recurrent layer
                dCdws += dot(self.S1[k].T, dCdZ1)
                
            
            "_____________________ MODIFIKASI DENGAN ADAM _____________________"
            
            "Parameter ADAM"
            
            t     = i
            beta1 = 0.9
            beta2 = 0.999
            eps   = 1E-8
            
            "Menghitung estimator momen m dan v"
            self.mw0 = beta1*self.mw0 + (1-beta1)*dCdw0
            self.mb0 = beta1*self.mb0 + (1-beta1)*dCdb0
            self.mw1 = beta1*self.mw1 + (1-beta1)*dCdw1
            self.mb1 = beta1*self.mb1 + (1-beta1)*dCdb1
            self.mws = beta1*self.mws + (1-beta1)*dCdws
            
            self.vw0 = beta2*self.vw0 + (1-beta2)*(dCdw0**2)
            self.vb0 = beta2*self.vb0 + (1-beta2)*(dCdb0**2)
            self.vw1 = beta2*self.vw1 + (1-beta2)*(dCdw1**2)
            self.vb1 = beta2*self.vb1 + (1-beta2)*(dCdb1**2)
            self.vws = beta2*self.vws + (1-beta2)*(dCdws**2)
                
            "Menghilangkan kebiasan m dan v"
            
            mw0_est = self.mw0/(1-beta1**(t+1))
            mb0_est = self.mb0/(1-beta1**(t+1))
            mw1_est = self.mw1/(1-beta1**(t+1))
            mb1_est = self.mb1/(1-beta1**(t+1))
            mws_est = self.mws/(1-beta1**(t+1))
            
            vw0_est = self.vw0/(1-beta2**(t+1))
            vb0_est = self.vb0/(1-beta2**(t+1))
            vw1_est = self.vw1/(1-beta2**(t+1))
            vb1_est = self.vb1/(1-beta2**(t+1))
            vws_est = self.vws/(1-beta2**(t+1))
                
            "Updating parameter W dan b"
            
            self.w0 -= (alpha/(np.sqrt(vw0_est)+eps))*mw0_est
            self.b0 -= (alpha/(np.sqrt(vb0_est)+eps))*mb0_est
            self.w1 -= (alpha/(np.sqrt(vw1_est)+eps))*mw1_est
            self.b1 -= (alpha/(np.sqrt(vb1_est)+eps))*mb1_est
            self.ws -= (alpha/(np.sqrt(vws_est)+eps))*mws_est
                
                
            "_____________________ MODIFIKASI DENGAN ADAM _____________________"

#### Pemakaian RNN untuk penaksiran harga saham

In [3]:
A  = pd.read_csv('ASII_10117080.csv')  #Data time series harian harga saham ASII
A['Adj Close'].fillna(A['Adj Close'].mean(), inplace=True)
A6 = np.array(A['Adj Close'])
B  = A6[:,newaxis]  #berupa matriks

Bmin = min(B)
Bmax = max(B)
b = (B-Bmin)/(Bmax-Bmin)

ts = 1   #Di literatur Time Series digunakan istilah 'lag' sebagai padanan istilah 'timestep' ini
xs = steps(b, ts)  #Dihasilkan matriks dengan 2 (= ts+1) kolom, kolom pertama menjadi variabel X
                   #dan kolom terkhir menjadi variabel 
x = xs[-800:,:]    #Ambil 800 observasi terakhir

#Data untuk training
Xtrain = x[0:680,:]   #Ambil 680 observasi yang pertama dan hilangkan kolom terakhir
ytrain = x[0:680:, -1:]  #Ambil 680 observasi yang pertama dan ambil kolom terakhir sebagai variabel y 

#Data untuk testing
Xtest = x[680:, :]   #ambil jumlah observasi sebanyak 120, hilangkan kolom terakhir (untuk y)
ytest = x[680:, -1:] #ambil kolom terakhir

#plt.plot(Xtrain, color = 'b')
#plt.show()

In [ ]:
nh     = 3  
alpha  = 0.001
epochs = 51

"Training"

rnn2 = RNN(Xtrain,nh,alpha,epochs)
tic = datetime.now()

for n in range(epochs):
    rnn2.learning()
    e = rnn2.ycap - ytrain
    sse = dot(e.T,e)/len(ytrain)
    toc = datetime.now()
            
    if n % ((epochs-1)/5) == 0:
        #print(f"\nPada epoch ke {n} diperoleh predicted vs actual:")
        #print(hstack((array(rnn2.ycap), ytrain.reshape(4,1))))
        print(f"Epoch ke {n}, SSE sebesar: {sse[0][0]:6.5f} dalam waktu {datetime.now()-tic}")

#rnn = RNN(Xtrain,nh,alpha,epochs)
#rnn.training()

plt.plot(ytrain, label = 'actual series')
plt.plot(rnn2.ycap, label = 'predicted series')
plt.legend()
plt.show()

Epoch ke 0, SSE sebesar: 0.31133 dalam waktu 0:00:00.913936


#### Test the model

In [ ]:
ytestcap = []

for i in range(len(Xtest)):
    S1 = [zeros((1, nh))]  
    S2 = []              
            
    "Forward propagation in time step"
    for k in range(rnn2.Ts):       
        A0 = Xtest[i,k]
        yk = Xtest[i,k + 1]
            
        # Forward pass, 1st layer
        Z1 = dot(A0,rnn2.w0) + rnn2.b0 + (S1[-1]@rnn2.ws) 
        A1 = tanh(Z1)
        S1.append(copy(A1))   
            
        # Forward pass, 2nd layer
        Z2 = A1@rnn2.w1 + rnn2.b1
        A2 = sigmoid(Z2)
        rnn2.S2.append(copy(A2))
                
    ytestcap.append(A2[0])

e = ytestcap - ytest
ssetest = dot(e.T,e)/len(ytest)
print('MSE training: %8.7f'%sse,'\nMSE testing : %7.7f'%ssetest)
      
plt.plot(ytest, label = 'actual series')
plt.plot(ytestcap, label = 'predicted series')
plt.legend()
plt.show()